In [4]:
import pandas as pd
from dash import dash, html, dcc, callback, Output, Input
import plotly.express as px

In [5]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [6]:
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': 'white', 'font-size': 40}),

    # TASK 1
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                 ],
                 value='ALL',
                 placeholder="Choose a launch site",
                 searchable=True
                 ),

    # TASK2
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # TASK 3
    html.P("Payload range (Kg):"),

    html.Div([
        dcc.RangeSlider(id='payload-slider',
                        min=0, max=10000, step=1000,
                        marks={0: '0', 10000: '10000'},
                        value=[min_payload, max_payload],
                    ),
        html.Div(id='output-container-range-slider')
    ]),

    # TASK 4
    html.Div([
        dcc.Graph(id='success-payload-scatter-chart')
    ]),
])

# TASK 2
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')]
)

def update_pie_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        grouped_df = filtered_df[filtered_df['class'] == 1].groupby('Launch Site').size().reset_index(name='Success Count')
        labels = grouped_df['Launch Site']
        values = grouped_df['Success Count']
        title = 'Total successful launches by site'
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) &
                                (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        success_count = filtered_df[filtered_df['class'] == 1]['class'].count()
        labels = [selected_site]
        values = [success_count]
        title = f'Total successful launches for {selected_site}'

    fig = px.pie(labels=labels, values=values, title=title)
    return fig

# TASK 4
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
)

def update_scatter_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) &
                                (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class')
    return fig

if __name__ == '__main__':
    app.run_server()